<a href="https://colab.research.google.com/github/Kaua1217/aula_Karlla/blob/main/atividade_transport_07_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala a biblioteca PuLP (necessário apenas na primeira execução no Colab)
!pip install pulp

# Importa as bibliotecas necessárias
from pulp import *

# Cria o problema de otimização (Minimização)
prob = LpProblem("Problema de Transporte", LpMinimize)

# Dados do problema: Custos unitários (R$/tonelada)
custos = {
    (1, 1): 4.00, (1, 2): 2.00, (1, 3): 6.00,
    (2, 1): 8.00, (2, 2): 2.00, (2, 3): 4.00,
    (3, 1): 4.00, (3, 2): 4.00, (3, 3): 4.00
}

# Dados do problema: Produção das Fábricas (Oferta)
producao = {
    1: 2000, 2: 2000, 3: 2000
}

# Dados do problema: Capacidade dos Depósitos (Demanda)
capacidade = {
    1: 3000, 2: 3000, 3: 1500
}

fabricas = list(producao.keys())
depositos = list(capacidade.keys())
rotas = [(f, d) for f in fabricas for d in depositos]

# Define as Variáveis de Decisão (x_ij: quantidade transportada)
x = LpVariable.dicts("Rota", rotas, lowBound=0, cat='Continuous')

# Define a Função Objetivo: Minimizar o Custo Total
prob += lpSum([custos[(f, d)] * x[(f, d)] for (f, d) in rotas]), "Custo Total de Transporte"

# Restrições de OFERTA (Fábricas): A produção total DEVE ser enviada (=)
for f in fabricas:
    prob += lpSum([x[(f, d)] for d in depositos]) == producao[f], f"Oferta_Fábrica_{f}"

# Restrições de CAPACIDADE (Depósitos): Carga recebida NÃO PODE exceder a capacidade (<=)
for d in depositos:
    prob += lpSum([x[(f, d)] for f in fabricas]) <= capacidade[d], f"Capacidade_Depósito_{d}"

# Resolve o problema
prob.solve()

# Apresenta os resultados
print(f"Status da Solução: {LpStatus[prob.status]}")
print("\n--- Resultados Ótimos ---")

# Verifica se a solução é ótima usando a constante correta: LpStatusOptimal
if prob.status == LpStatusOptimal:
    # Valores das variáveis de decisão (alocação)
    print("\nAlocação (toneladas):")
    for (f, d) in rotas:
        if x[(f, d)].varValue > 0:
            print(f"  Fábrica {f} -> Depósito {d}: {x[(f, d)].varValue:,.0f} t (Custo R${custos[(f, d)]:.2f}/t)")

    # Valor da Função Objetivo (Custo Mínimo Total)
    custo_total = value(prob.objective)
    print(f"\nCUSTO MÍNIMO TOTAL: R$ {custo_total:,.2f}")

    # Análise das restrições de Capacidade (Demanda Atendida)
    print("\nAnálise de Capacidade nos Depósitos (Demanda Atendida):")
    for d in depositos:
        recebido = sum(x[(f, d)].varValue for f in fabricas)
        capacidade_d = capacidade[d]
        print(f"  Depósito {d} - Recebido: {recebido:,.0f} t (Capacidade: {capacidade_d:,.0f} t)")

else:
    print("O problema não encontrou uma solução ótima.")

Status da Solução: Optimal

--- Resultados Ótimos ---

Alocação (toneladas):
  Fábrica 1 -> Depósito 2: 2,000 t (Custo R$2.00/t)
  Fábrica 2 -> Depósito 2: 1,000 t (Custo R$2.00/t)
  Fábrica 2 -> Depósito 3: 1,000 t (Custo R$4.00/t)
  Fábrica 3 -> Depósito 1: 2,000 t (Custo R$4.00/t)

CUSTO MÍNIMO TOTAL: R$ 18,000.00

Análise de Capacidade nos Depósitos (Demanda Atendida):
  Depósito 1 - Recebido: 2,000 t (Capacidade: 3,000 t)
  Depósito 2 - Recebido: 3,000 t (Capacidade: 3,000 t)
  Depósito 3 - Recebido: 1,000 t (Capacidade: 1,500 t)
